In [3]:
import os
import string
import polars as pl
import altair as alt

pl.Config.set_tbl_rows(300)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [5]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    (pl.col('datum').dt.year() >= 1918)
    & 
    (~pl.col('komora_komplet').str.contains('(?i)senát')) 
    & 
    (~pl.col('komora_komplet').str.contains('(?i)ústavo')) 
    & 
    (~pl.col('komora_komplet').str.contains('(?i)společn'))
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).collect()

In [7]:
df.filter(pl.col("datum").dt.year() == 1930)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Předseda""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""Malypetr.""",1,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,0
"""Místopředsedové""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""dr. Lukavský, Roudnický, Stivín, Špatný, Taub, Zierhut.""",2,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,6
"""Zapisovatelé""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""Langr, Rýpar. 223 poslanců podle presenční listiny. Zástupci vlády: min. předseda Udržal; ministři Bechyně, Bradáč, dr Czech, dr Dérer, inž. Dostálek,…",3,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,48
"""Předseda (zvoní)""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""Zahajuji 8. schůzi posl. sněmovny. Dovolené dal jsem na dnešní schůzi posl. Šcereckému, Vallo, Kaliňákovi, Čižinské, Doričovi, Hodinové, Petrovi, Brož…",4,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,97
"""Zástupce sněm. tajemníka dr Mikyška (čte)""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""Dotazy: posl. Bezděka, dr Daňka a druhů ministrům zemědělství a financí o zřízení lihovaru v Řečici na Novoměstsku (č. D 44-III); posl. dr Patejdla mi…",5,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,179
"""Předseda""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""Počátkem schůze byl tiskem rozdán vládní návrh.""",6,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,6
"""Zástupce sněm. tajemníka dr Mikyška (čte)""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""40. Vládní návrh, kterým se předkládá Národnímu shromáždění k projevu souhlasu dohoda mezi republikou Československou a Dánskem, sjednaná výměnou not …",7,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,49
"""Předseda""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""Počátkem schůze byly tiskem rozdány zápisy o 2. až 7. schůzi posl. sněmovny, proti nimž nebylo námitek podle §u 73 jedn. řádu. Počátkem schůze byla ti…",8,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,45
"""Zástupce sněm. tajemníka dr Mikyška (čte)""","""1929ns_ps_stenprot_008schuz_s008001.htm""",null,"""37. Návrh posl. B. Procházky, Bergmanna, Tučného a druhů na změnu zákona ze dne 24. června 1926, č. 103 Sb. z. a n., o úpravě platových a některých sl…",9,"""ps""",1929,8,1,1930-01-08,"""NS RČS 1929-1935, Poslanecká sněmovna""","""8. schůze""",true,true,true,null,33


In [22]:
df.filter(pl.col("datum").dt.year() == 1930).filter(pl.col("text").str.contains("ľ"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Zástupce sněm. tajemníka dr Záděra (čte)""","""1929ns_ps_stenprot_009schuz_s009001.htm""",null,"""V ustavujících schůzích dne 9. ledna 1930 zvolil výbor: Branný: předsedou posl. Davida; místopředsedy posl. Pekárka, Machníka, Špačka; zapisovatelem p…",3,"""ps""",1929,9,1,1930-01-16,"""NS RČS 1929-1935, Poslanecká sněmovna""","""9. schůze""",true,true,true,null,492
"""Zástupce sněm. tajemníka dr Záděra (čte)""","""1929ns_ps_stenprot_009schuz_s009001.htm""",null,"""27. Interpelace: I. posl. Šeby, Ot. Svobody a druhů předsedovi vlády ve vlastnosti dozorčího orgánu nad stát. pozemkovým úřadem o přídělu hřišť ze zab…",14,"""ps""",1929,9,1,1930-01-16,"""NS RČS 1929-1935, Poslanecká sněmovna""","""9. schůze""",true,true,true,null,784
"""Zástupce sněm. tajemníka dr Záděra (čte)""","""1929ns_ps_stenprot_009schuz_s009006.htm""",null,"""22. Návrh posl. Petroviča, Kaliňáka, Zajíca a druhov na vydanie zákona o predlžení zákona zo dňa 26. apríla 1924, č. 87 Sb. z. a n., o úprave uživania…",20,"""ps""",1929,9,6,1930-01-16,"""NS RČS 1929-1935, Poslanecká sněmovna""","""9. schůze""",true,true,true,null,77
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1929ns_ps_stenprot_010schuz_s010001.htm""",null,"""Naléhavé interpelace: posl. Köhlera a druhů ministru soc. péče o pensijním pojištění soukromých zaměstnanců, posl. Hlinku a druhov vláde o poškodzovan…",9,"""ps""",1929,10,1,1930-01-23,"""NS RČS 1929-1935, Poslanecká sněmovna""","""10. schůze""",true,true,true,null,41
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1929ns_ps_stenprot_010schuz_s010001.htm""",null,"""Dotazy: posl. Sladkého: ministru školství a nár. osvěty o urychlení výstavby státních hraničářských škol na Příborsku, Novojicku a na Hranicku na Mora…",11,"""ps""",1929,10,1,1930-01-23,"""NS RČS 1929-1935, Poslanecká sněmovna""","""10. schůze""",true,true,true,null,238
"""Zástupce sněm. tajemníka dr Záděra (čte)""","""1929ns_ps_stenprot_012schuz_s012001.htm""",null,"""114. Návrh posl. Kleina, Fr. Svobody, Bečko a soudr., aby byl vydán zákon o pracovním poměru soukromých zaměstnanců. 121. Návrh posl. Köhlera a druhů …",16,"""ps""",1929,12,1,1930-01-30,"""NS RČS 1929-1935, Poslanecká sněmovna""","""12. schůze""",true,true,true,null,135
"""Zástupce sněm. tajemníka dr Záděra (čte)""","""1929ns_ps_stenprot_012schuz_s012006.htm""",null,"""15. Naléhavá interpelace posl. Bergmanna, B. Procházky, Pechmanové, dr Patejdla a druhů vládě o činnosti komise pro normalisaci a systemisaci služební…",4,"""ps""",1929,12,6,1930-01-30,"""NS RČS 1929-1935, Poslanecká sněmovna""","""12. schůze""",true,true,true,null,254
"""Posl. dr Szüllö (maďarsky)""","""1929ns_ps_stenprot_013schuz_s013005.htm""",null,"""Ctená snemovňa! Keď prečítal som expozé pána ministra Beneša, prišla mi na um epizoda, kde Ľudvik XVI. vyslal generála Dumourieza s určitou diplomatic…",3,"""ps""",1929,13,5,1930-02-03,"""NS RČS 1929-1935, Poslanecká sněmovna""","""13. schůze""",true,true,true,null,2341
"""Posl. dr Szüllö (maďarsky)""","""1929ns_ps_stenprot_013schuz_s013006.htm""",null,"""Je na čase, aby vedenie zahraničných vecí dostalo iný smer, aby minister zahraničia dovtípil sa toho, že nie je zahraničným ministrom svetovej ríše, a…",1,"""ps""",1929,13,6,1930-02-03,"""NS RČS 1929-1935, Poslanecká sněmovna""","""13. schůze""",true,true,false,null,203


In [18]:
from langdetect import detect
from langdetect import detect_langs

In [24]:
detect_langs("Slávna snemovňa! Akokoľvek posudzovali by sme reč ministra dr Beneša, jedno jej nemožno uprieť, a to, že svojím obsahom i svojou formou československú")

[sk:0.9999954857328808]

In [34]:
def jazyk(retezec):
    try:
        return detect(retezec)
    except Exception as e:
        print(retezec)
        print(e)
        print("***")
        return None

In [40]:
def jazyky(retezec):
    try:
        return detect_languages(retezec)
    except Exception as e:
        print(retezec)
        print(e)
        print("***")
        return None

In [44]:
jazyky("Pak bych vás chtěla poprosit, abyste mi začali posílat své požadavky na ubytování, kapacity jsou omezené.")

Pak bych vás chtěla poprosit, abyste mi začali posílat své požadavky na ubytování, kapacity jsou omezené.
name 'detect_languages' is not defined
***


In [42]:
df.filter(pl.col("datum").dt.year() == 1925).with_columns(pl.col("text").map_elements(jazyky).alias("jazyky"))

C:\Users\micha\AppData\Local\Temp\ipykernel_28420\833287004.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df.filter(pl.col("datum").dt.year() == 1925).with_columns(pl.col("text").map_elements(jazyky).alias("jazyky"))
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,jazyky
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str
"""Výboru živnostenskému""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Začátek schůze ve 3 hod. 50 min. odpol. Přítomni:""",1,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,false,null,8,null
"""Předseda""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Tomášek. Místopředsedové: dr. Czech, dr. inž. Botto, Buříval, dr. Hruban. Zapisovatelé: Benda, Roudnický. 211 poslanců podle presenční listiny. Zástup…",2,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,264,null
"""Sněm. tajemník dr. Říha (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""""Slibuji, že budu věren republice Československé a že budu zachovávati zákony a mandát svůj zastávati podle svého nejlepšího vědomí a svědomí.""""",3,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,20,null
"""Předseda""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Pan posl. Světlík.""",4,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,2,null
"""Posl. Světlík (podávaje předsedovi ruku)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Slibuji. (Poslanci usedají.) Předseda (zvoní): Předseda vlády oznámil: přípisem ze dne 3. ledna 1925, čís. 7/1076/25/S, změnu ve vzájemném zastupování…",5,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,88,null
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Volební soud v Praze přípisem ze dne 19. prosince 1924, čís. 606/24, sdělil svůj nález z téhož dne a téhož čísla, kterým jednak jako nepřípustné, jedn…",6,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,135,null
"""Dále došel přípis ze dne 20. prosince 1924, čís. 607/24, že se ověřují volby poslanců""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""dr. Františka Samka, generálního sekretáře Obchodní a živnostenské komory v Praze, nastoupivšího místo posl. Václava Veverky, jenž se mandátu vzdal; R…",7,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,62,null
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Posl. Rychtera oznámil přípisem dne 30. prosince 1924, že vystoupil z klubu poslanců republikánské strany zemědělského a malorolnického lidu. Také klu…",8,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,156,null
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Předseda senátu sdělil přípisem ze dne 12. prosince 1924, že senát projednal ve 237., 238. a 239. schůzi dne 10., 11. a 12. prosince 1924 zkráceným ří…",9,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,727,null


In [38]:
df.filter(pl.col("datum").dt.year() == 1925).with_columns(pl.col("text").map_elements(jazyk, return_dtype=pl.String).alias("jazyk"))


No features in text.
***


mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,jazyk
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str
"""Výboru živnostenskému""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Začátek schůze ve 3 hod. 50 min. odpol. Přítomni:""",1,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,false,null,8,"""cs"""
"""Předseda""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Tomášek. Místopředsedové: dr. Czech, dr. inž. Botto, Buříval, dr. Hruban. Zapisovatelé: Benda, Roudnický. 211 poslanců podle presenční listiny. Zástup…",2,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,264,"""cs"""
"""Sněm. tajemník dr. Říha (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""""Slibuji, že budu věren republice Československé a že budu zachovávati zákony a mandát svůj zastávati podle svého nejlepšího vědomí a svědomí.""""",3,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,20,"""cs"""
"""Předseda""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Pan posl. Světlík.""",4,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,2,"""cs"""
"""Posl. Světlík (podávaje předsedovi ruku)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Slibuji. (Poslanci usedají.) Předseda (zvoní): Předseda vlády oznámil: přípisem ze dne 3. ledna 1925, čís. 7/1076/25/S, změnu ve vzájemném zastupování…",5,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,88,"""cs"""
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Volební soud v Praze přípisem ze dne 19. prosince 1924, čís. 606/24, sdělil svůj nález z téhož dne a téhož čísla, kterým jednak jako nepřípustné, jedn…",6,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,135,"""cs"""
"""Dále došel přípis ze dne 20. prosince 1924, čís. 607/24, že se ověřují volby poslanců""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""dr. Františka Samka, generálního sekretáře Obchodní a živnostenské komory v Praze, nastoupivšího místo posl. Václava Veverky, jenž se mandátu vzdal; R…",7,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,62,"""cs"""
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Posl. Rychtera oznámil přípisem dne 30. prosince 1924, že vystoupil z klubu poslanců republikánské strany zemědělského a malorolnického lidu. Také klu…",8,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,156,"""cs"""
"""Zástupce sněm. tajemníka Nebuška (čte)""","""1920ns_ps_stenprot_320schuz_s320001.htm""",null,"""Předseda senátu sdělil přípisem ze dne 12. prosince 1924, že senát projednal ve 237., 238. a 239. schůzi dne 10., 11. a 12. prosince 1924 zkráceným ří…",9,"""ps""",1920,320,1,1925-01-29,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""320. schůze""",true,true,true,null,727,"""cs"""
